<a href="https://colab.research.google.com/github/fvillena/dcc-ia-nlp/blob/master/8-transformers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Actividad utilizando Transformers

### Librerías necesarias


In [ ]:
!pip install transformers

In [ ]:
from transformers import pipeline, AutoTokenizer
import pandas as pd

# Pipelines

En esta actividad vamos a utilizar Transformers para extraer información del texto. En particular vamos a utilizar los pipelines de:


*   Named Entity Recognition
*   Question Answering



Cargamos los modelos pre-entrenados disponibles en HuggingFace

In [ ]:
# Transformer NER
nlp_ner = pipeline("ner", model="mrm8488/bert-spanish-cased-finetuned-ner",
    tokenizer=('mrm8488/bert-spanish-cased-finetuned-ner', {"use_fast": False})
    )
# Transformer QA
nlp_qa = pipeline('question-answering', 
        model='mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es',
        tokenizer=('mrm8488/distill-bert-base-spanish-wwm-cased-finetuned-spa-squad2-es', {"use_fast": False})
    )
# Lo necesitamos para sacar el texto asociado a cada token. La predicción de ner viene con [UNK] cuando no reconoce el
# como un token
tok = AutoTokenizer.from_pretrained('mrm8488/bert-spanish-cased-finetuned-ner')

# Named Entity Recognition
El pipeline NER detecta diferentes entidades en el texto analizado. Ejemplo:

In [ ]:
texto = 'Carlos es estudiante de medicina, vive en Santiago hace 3 años'
df_result = pd.DataFrame(nlp_ner(texto))
print(df_result)

  entity     score  index      word start   end
0  B-PER  0.998058      1    Carlos  None  None
1  B-LOC  0.999609      9  Santiago  None  None


Vemos que el pipeline detecta:
*   Una entidad de persona en el token índice 1
*   Una entidad de localidad en el token índice 9

Como se puede observar en el resultado anterior, el pipeline también devuelve la palabra asociada al token (columna *word*)

Puede suceder que entidad detectada corresponda a la concatenación de 2 o más tokens. Ejemplo:

In [ ]:
texto = 'Carlitos es estudiante de medicina, vive en Santiago hace 3 años'
df_result = pd.DataFrame(nlp_ner(texto))
print(df_result)

  entity     score  index      word start   end
0  B-PER  0.999317      1      Carl  None  None
1  I-PER  0.709062      2    ##itos  None  None
2  B-LOC  0.999640     10  Santiago  None  None


En este caso, la persona *Carlitos* está compuestas por 2 tokens (*Carl* y *##itos*)

También puede suceder que el string asociado a la persona, no se corresponda con ningún token del tokenizador, pero de todas formas el pipeline detecta la entidad asociada. Ejemplo:

In [ ]:
texto = 'Juan es estudiante de medicina, vive en Santiago hace 3 años'
df_result = pd.DataFrame(nlp_ner(texto))
print(df_result)

  entity     score  index      word start   end
0  B-PER  0.998483      1     [UNK]  None  None
1  B-LOC  0.999622      9  Santiago  None  None


Para poder obtener el texto asociado a esa entidad debemos utilizar el mapeo realizado por el tokenizador de la siguiente manera.

In [ ]:
# Mapping tiene una lista con índices (start, end) correspondiente al texto de cada token, en el mismo orden
mapping = tok(texto, return_offsets_mapping=True)['offset_mapping']
# Llenamos las columnas start, end y text con el texto correspondiente a cada token
for i, row in df_result.iterrows():
    df_result.loc[i, 'text'] = texto[mapping[row['index']][0]:mapping[row['index']][1]]
    df_result.loc[i, 'start'] = mapping[row['index']][0]
    df_result.loc[i, 'end'] = mapping[row['index']][1]
print(df_result)

  entity     score  index      word start end      text
0  B-PER  0.998483      1     [UNK]     0   4      Juan
1  B-LOC  0.999622      9  Santiago    40  48  Santiago


De esta manera, podemos obtener el comienzo y fin de cada string que fué detectado como una entidad.

# Question Answering
El pipeline QA permite realizar preguntas en un determinado context (texto). en los siguientes ejemplos se muestra modo utlizarlo.

In [ ]:
texto = 'Carlos es estudiante de medicina, vive en Santiago hace 3 años'

question = '¿Quién Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 0.9089308977127075, 'start': 10, 'end': 33, 'answer': 'estudiante de medicina,'}


In [ ]:
question = '¿Dónde vive Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 0.9807462692260742, 'start': 42, 'end': 50, 'answer': 'Santiago'}


In [ ]:
question = '¿De que color es el gato de Carlos?'
respuesta = nlp_qa({'question': question, 'context': texto})
print(respuesta)

{'score': 1.5464765965589322e-06, 'start': 58, 'end': 61, 'answer': 'años'}


Hay que tener en cuenta que el pipeline **siempre** devuelve una respuesta. Utilizar su score asociado puede resultar últil para determinar qué tan buena es la misma.

# Actividad
Se tiene un conjunto de documentos correspondientes a contratos de locación de una inmobiliaria. Para cada contrato, se pretende extraer de manera automática:
* Las personas que intervienen en el contrato
* Determinar quién es el locador y el locatario
* Determinar la dirección del inmueble alquilado